In [4]:
import gym
from tqdm import tqdm
import numpy as np

import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [5]:
from stable_baselines3 import PPO, A2C, SAC, TD3
from stable_baselines3.common.evaluation import evaluate_policy

In [6]:
# continuous actions
# env_id = "LunarLanderContinuous-v2"

# discrete actions
env_id = "CartPole-v1"

In [7]:
env = gym.make(env_id)

# train expert policy

In [8]:
ppo_expert = PPO('MlpPolicy', env_id, verbose=1, create_eval_env=True)
ppo_expert.learn(total_timesteps=3e4, eval_freq=10000)
ppo_expert.save("ppo_expert")

Using cuda device
Creating environment from the given name, wrapped in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.7     |
|    ep_rew_mean     | 23.7     |
| time/              |          |
|    fps             | 1241     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 26.2        |
|    ep_rew_mean          | 26.2        |
| time/                   |             |
|    fps                  | 895         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.006888212 |
|    clip_fraction        | 0.0938      |
|    clip_range           | 0.2         |
|    entropy_loss         | 

In [9]:
mean_reward, std_reward = evaluate_policy(ppo_expert, env, n_eval_episodes=10)

print(f"Mean reward = {mean_reward} +/- {std_reward}")

Mean reward = 500.0 +/- 0.0


# create student policy
### it may be diffrent algorithm 

In [10]:
ppo_student = PPO('MlpPolicy', env_id, verbose=1)


Using cuda device
Creating environment from the given name, wrapped in a DummyVecEnv.


In [11]:
# only valid for continuous actions
# sac_student = SAC('MlpPolicy', env_id, verbose=1, policy_kwargs=dict(net_arch=[64, 64]))

### number of samples from expert to be used as "Data"

In [12]:
num_interactions = int(4e4)

In [14]:
if isinstance(env.action_space, gym.spaces.Box):
  expert_observations = np.empty((num_interactions,) + env.observation_space.shape)
  expert_actions = np.empty((num_interactions,) + (env.action_space.shape[0],))

else:
  expert_observations = np.empty((num_interactions,) + env.observation_space.shape)
  expert_actions = np.empty((num_interactions,) + env.action_space.shape)

obs = env.reset()

for i in tqdm(range(num_interactions)):
    action, _ = ppo_expert.predict(obs, deterministic=True)
    expert_observations[i] = obs
    expert_actions[i] = action
    obs, reward, done, info = env.step(action)
    if done:
        obs = env.reset()

np.savez_compressed(
    "expert_data",
    expert_actions=expert_actions,
    expert_observations=expert_observations,
)

100%|██████████| 40000/40000 [00:23<00:00, 1684.34it/s]


### now reuse the generated expert data to train the student policy

In [15]:
from torch.utils.data.dataset import Dataset, random_split

In [16]:
class ExpertDataSet(Dataset):
    def __init__(self, expert_observations, expert_actions):
        self.observations = expert_observations
        self.actions = expert_actions
        
    def __getitem__(self, index):
        return (self.observations[index], self.actions[index])

    def __len__(self):
        return len(self.observations)

In [17]:
expert_dataset = ExpertDataSet(expert_observations, expert_actions)

train_size = int(0.8 * len(expert_dataset))

test_size = len(expert_dataset) - train_size

train_expert_dataset, test_expert_dataset = random_split(
    expert_dataset, [train_size, test_size]
)

In [18]:
print("test_expert_dataset: ", len(test_expert_dataset))
print("train_expert_dataset: ", len(train_expert_dataset))

test_expert_dataset:  8000
train_expert_dataset:  32000


In [19]:
def pretrain_agent(
    student,
    batch_size=64,
    epochs=1000,
    scheduler_gamma=0.7,
    learning_rate=1.0,
    log_interval=100,
    no_cuda=True,
    seed=1,
    test_batch_size=64,
):
    use_cuda = not no_cuda and th.cuda.is_available()
    th.manual_seed(seed)
    device = th.device("cuda" if use_cuda else "cpu")
    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    if isinstance(env.action_space, gym.spaces.Box):
      criterion = nn.MSELoss()
    else:
      criterion = nn.CrossEntropyLoss()

    # Extract initial policy
    model = student.policy.to(device)

    def train(model, device, train_loader, optimizer):
        model.train()

        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()

            if isinstance(env.action_space, gym.spaces.Box):
              # A2C/PPO policy outputs actions, values, log_prob
              # SAC/TD3 policy outputs actions only
              if isinstance(student, (A2C, PPO)):
                action, _, _ = model(data)
              else:
                # SAC/TD3:
                action = model(data)
              action_prediction = action.double()
            else:
              # Retrieve the logits for A2C/PPO when using discrete actions
              latent_pi, _, _ = model._get_latent(data)
              logits = model.action_net(latent_pi)
              action_prediction = logits
              target = target.long()

            loss = criterion(action_prediction, target)
            loss.backward()
            optimizer.step()
            if batch_idx % log_interval == 0:
                print(
                    "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch,
                        batch_idx * len(data),
                        len(train_loader.dataset),
                        100.0 * batch_idx / len(train_loader),
                        loss.item(),
                    )
                )
    def test(model, device, test_loader):
        model.eval()
        test_loss = 0
        with th.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)

                if isinstance(env.action_space, gym.spaces.Box):
                  # A2C/PPO policy outputs actions, values, log_prob
                  # SAC/TD3 policy outputs actions only
                  if isinstance(student, (A2C, PPO)):
                    action, _, _ = model(data)
                  else:
                    # SAC/TD3:
                    action = model(data)
                  action_prediction = action.double()
                else:
                  # Retrieve the logits for A2C/PPO when using discrete actions
                  latent_pi, _, _ = model._get_latent(data)
                  logits = model.action_net(latent_pi)
                  action_prediction = logits
                  target = target.long()

                test_loss = criterion(action_prediction, target)
        test_loss /= len(test_loader.dataset)
        print(f"Test set: Average loss: {test_loss:.4f}")

    # Here, we use PyTorch `DataLoader` to our load previously created `ExpertDataset` for training
    # and testing
    train_loader = th.utils.data.DataLoader(
        dataset=train_expert_dataset, batch_size=batch_size, shuffle=True, **kwargs
    )
    test_loader = th.utils.data.DataLoader(
        dataset=test_expert_dataset, batch_size=test_batch_size, shuffle=True, **kwargs,
    )

    # Define an Optimizer and a learning rate schedule.
    optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=1, gamma=scheduler_gamma)

    # Now we are finally ready to train the policy model.
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer)
        test(model, device, test_loader)
        scheduler.step()

    # Implant the trained policy network back into the RL student agent
    ppo_student.policy = model

In [20]:
### evaluate the agent before pretraining

In [30]:
mean_reward, std_reward = evaluate_policy(ppo_student, env, n_eval_episodes=10)

print(f"Mean reward = {mean_reward} +/- {std_reward}")


Mean reward = 8.9 +/- 0.5385164807134504


### pretraining

In [32]:
pretrain_agent(
    ppo_student,
    epochs=3,
    scheduler_gamma=0.7,
    learning_rate=1.0,
    log_interval=100,
    no_cuda=True,
    seed=1,
    batch_size=64,
    test_batch_size=1000,
)
ppo_student.save("a2c_student")

Train Epoch: 1 [0/32000 (0%)]	Loss: 0.693642
Train Epoch: 1 [6400/32000 (20%)]	Loss: 0.374572
Train Epoch: 1 [12800/32000 (40%)]	Loss: 0.321954
Train Epoch: 1 [19200/32000 (60%)]	Loss: 0.325045
Train Epoch: 1 [25600/32000 (80%)]	Loss: 0.161926
Test set: Average loss: 0.0000
Train Epoch: 2 [0/32000 (0%)]	Loss: 0.210440
Train Epoch: 2 [6400/32000 (20%)]	Loss: 0.245195
Train Epoch: 2 [12800/32000 (40%)]	Loss: 0.218046
Train Epoch: 2 [19200/32000 (60%)]	Loss: 0.308945
Train Epoch: 2 [25600/32000 (80%)]	Loss: 0.227988
Test set: Average loss: 0.0000
Train Epoch: 3 [0/32000 (0%)]	Loss: 0.181285
Train Epoch: 3 [6400/32000 (20%)]	Loss: 0.268268
Train Epoch: 3 [12800/32000 (40%)]	Loss: 0.154440
Train Epoch: 3 [19200/32000 (60%)]	Loss: 0.122000
Train Epoch: 3 [25600/32000 (80%)]	Loss: 0.185891
Test set: Average loss: 0.0000


In [34]:
mean_reward, std_reward = evaluate_policy(ppo_student, env, n_eval_episodes=10)

print(f"Mean reward = {mean_reward} +/- {std_reward}")

RuntimeError: CUDA error: an illegal memory access was encountered